<a href="https://colab.research.google.com/github/Skorobogatchenko2004/sumo/blob/main/BayesML_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа № 1
### Изучение принципов работы байесовских сетей c использованием фреймворка Pyro и библиотеки ArviZ

#### Задачи работы:
1. Познакомиться с байесовским моделированием и организацией байесовских сетей.
2. Изучить принципы создания байесовских моделей для решения стандартных задач машинного обучения.
3. Познакомиться с байесовским подходом к заполнению пропусков в данных.
4. Получить навыки создания и обучения моделей, а также получения результатов предсказания на их основе, с использованием фреймворка Pyro.
5. Получить навыки интерпретации байесовских моделей с использованием библиотеки ArviZ.

Устанавливаем фреймворк вероятностного программирования [Pyro](https://pyro.ai/):

In [ ]:
!pip install pyro-ppl

Подключаем библиотеки анализа и визуализации данных, необходимые в процессе работы.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import torch
import pyro
from pyro import distributions as dist
from pyro.infer import MCMC, NUTS, Predictive
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

%matplotlib inline

Уточняем версию фреймворка Pyro.

In [ ]:
pyro.__version__

В работе анализируется датасет [Credit Risk Dataset](https://www.kaggle.com/datasets/laotse/credit-risk-dataset), который содержит информацию о клиентах, получивших кредит.

Данный датасет был разделен на две части (train и predict), предназначенные для обучения байесовской модели и получения по ней предсказаний соответственно.

Скачиваем [тренировочный](https://drive.google.com/file/d/1HnBdQodlt3MyAd6NVw0qOKxQYVbc7X8s/view?usp=sharing) и [тестовый](https://drive.google.com/file/d/1n9XjptSj8j4BL1gFwxXBFN69MtjsSPQE/view?usp=sharing) датасеты в локальную папку.

Загрузим датасет для обучения модели и посмотрим на него.

In [ ]:
!gdown 1HnBdQodlt3MyAd6NVw0qOKxQYVbc7X8s
!gdown 1n9XjptSj8j4BL1gFwxXBFN69MtjsSPQE

Описание столбцов датасета представлено ниже.

In [ ]:
df_train_name = "/content/credit_risk_dataset_train.csv"
df_train = pd.read_csv(df_train_name)
df_train

|         **Признак**        |                    **Значение**                   |
|:--------------------------:|:-------------------------------------------------:|
| person_age                 | Возраст                                           |
| person_income              | Годовой доход                                     |
| person_home_ownership      | Владение жильем                                   |
| person_emp_length          | Продолжительность трудовой деятельности (в годах) |
| loan_intent                | Намерения по кредиту                              |
| loan_grade                 | Оценка кредита                                    |
| loan_amnt                  | Сумма кредита                                     |
| loan_int_rate              | Процентная ставка                                 |
| loan_status                | Статус кредита (0 - не дефолт 1 - дефолт)         |
| loan_percent_income        | Процентный доход                                  |
| cb_person_default_on_file  | Исторический дефолт                               |
| cb_preson_cred_hist_length | Длина кредитной истории                           |

В датасете наблюдаются признаки:

* **количественные**: возраст, годовой доход, продолжительность трудовой деятельности, сумма кредита, процентная ставка, процентный доход, длина кредитной истории.
* **категориальные**: владение жильём, намерения по кредиту, оценка кредита, статус кредита, исторический дефолт.

Взглянем на формат столбцов датасета.

In [ ]:
df_train.info()

с целью упрощения анализа, оставим в датасете только некоторые столбцы.

In [ ]:
df_train = df_train[
    ["person_age", "person_income", "person_home_ownership", "loan_intent",
     "loan_amnt", "loan_int_rate", "cb_person_default_on_file",
     "loan_status"]
]

Посмотрим, содержат ли столбцы пропущенные значения.

In [ ]:
df_train.isnull().sum()

Видно, процентная ставка указана не для всех клиентов. Пропущенные значения будем заполнять в дальнейшем с применением байесовской модели.

Посмотрим на статистику данных.

In [ ]:
df_train.describe()

Максимальный возраст клиента в 144 года вызывает сомнения...

Выведем несколько клиентов, отсортированных по возрасту в порядке убывания.

In [ ]:
df_train.sort_values(by="person_age", ascending=False).head(10)

Видно, что клиентов с подозрительным значением возраста не так много. Удалим их из датасета.

In [ ]:
df_train = df_train[df_train["person_age"] < 100]
df_train.describe()

Посмотрим на количество значений категориальных признаков.

In [ ]:
for col in ["person_home_ownership",
            "loan_intent",
            "cb_person_default_on_file",
            "loan_status"]:
    print(df_train[col].value_counts(), end="\n\n")

Количество различных значений категориальных столбцов невелико, специальным образом обрабатывать их не нужно.

Построим гистограммы категориальных признаков.

In [ ]:
df_train["person_age"].hist(bins=100)

In [ ]:
df_train["person_income"].hist(bins=100)

In [ ]:
df_train[df_train["person_income"] < 500000]["person_income"].hist(bins=100)

In [ ]:
df_train["loan_amnt"].hist(bins=100)

In [ ]:
df_train["loan_int_rate"].hist(bins=100)

Приведем типы столбцов датасета к корректным.

In [ ]:
person_home_ownership_cat = pd.CategoricalDtype(
    categories=["RENT", "MORTGAGE", "OWN", "OTHER"],
    ordered=True
)

loan_intent_cat = pd.CategoricalDtype(
    categories=["EDUCATION", "MEDICAL", "VENTURE", "PERSONAL",
                "DEBTCONSOLIDATION", "HOMEIMPROVEMENT"],
    ordered=True
)

cb_person_default_on_file_cat = pd.CategoricalDtype(
    categories=["N", "Y"],
    ordered=True
)

In [ ]:
df_train = df_train.astype(
    {
        "person_age": np.float64,
        "person_income": np.float64,
        "person_home_ownership": person_home_ownership_cat,
        "loan_intent": loan_intent_cat,
        "loan_amnt": np.float64,
        "loan_int_rate": np.float64,
        "cb_person_default_on_file": cb_person_default_on_file_cat,
        "loan_status": np.int64
    }

)

df_train.info()

Выполним кодирование категориальных признаков и стандартизацию количественных.

Признак с процентной ставкой, пустые значения в котором будем заполнять с помощью байесовской модели, оставим без изменений.

In [ ]:
num_features = ["person_age", "person_income", "loan_amnt"]
cat_features = ["person_home_ownership", "loan_intent",
                "cb_person_default_on_file"]

column_transformer = ColumnTransformer([
    ("cat_transformer", OrdinalEncoder(dtype='int'), cat_features),
    ("num_transformer", StandardScaler(), num_features)
], remainder='passthrough', verbose_feature_names_out=False)

In [ ]:
column_transformer.set_output(transform="pandas")
df_train = column_transformer.fit_transform(df_train)
df_train

Поставим __задачу классификации__ следующим образом: предсказать вероятность возврата кредита клиентом по его характеристикам, представленным в датасете. Одновременно с решением задачи классификации следует заполнить пропуски в столбце "Процентная ставка". Требуется визуализовать априорные распределения коэффициентов байесовской модели, распределение целевой переменной для произвольного объекта тестового датасета.

Байесовскую модель зададим следующим образом. Курсивом обозначены случайные переменные, прямым шрифтом -- признаки из датасета.

$b\_person\_age \sim \mathcal{N}(0,\,1)$

$b\_person\_income \sim \mathcal{N}(0,\,1)$

$b\_loan\_amnt \sim \mathcal{N}(0,\,1)$

$b\_loan\_int\_rate \sim \mathcal{N}(0,\,1)$

$b\_person\_home\_ownership_i \sim \mathcal{N}(0,\,1), \: i \in [0,\,3]$

$b\_loan\_intent_i \sim \mathcal{N}(0,\,1), \: i \in [0,\,5]$

$b\_cb\_person\_default\_on\_file_i \sim \mathcal{N}(0,\,1), \: i \in [0,\,1]$

$mu\_loan\_int\_rate \sim \mathcal{N}(0,\,1)$

$std\_loan\_int\_rate \sim |\mathcal{N}(0,\,1)|$

$impute\_loan\_int\_rate \sim \mathcal{N}(mu\_loan\_int\_rate,\,std\_loan\_int\_rate)$

$loan\_int\_rate \sim \mathcal{N}(mu\_loan\_int\_rate,\,std\_loan\_int\_rate)$


$logits_i = b\_person\_age \cdot \text{person_age} + b\_person\_income \cdot \text{person_income} + b\_loan\_amnt \cdot \text{loan_amnt} + b\_person\_home\_ownership \cdot \text{person_home_ownership} + b\_loan\_intent \cdot \text{loan_intent} + b\_cb\_person\_default\_on\_file \cdot \text{b_cb_person_default_on_file} + b\_loan\_int\_rate \cdot
  \begin{cases}
    \text{loan_int_rate}_i     & \text{if loan_int_rate$_i$ is not None}, \\
    impute\_loan\_int\_rate & \text{otherwise}.
  \end{cases}
$

\begin{cases}
    \text{loan_int_rate} = \sum_{i=1}^N \mathcal{N}(mu\_loan\_int\_rate,\,std\_loan\_int\_rate)    & \text{if loan_int_rate$_i$ is not None}, \\
    impute\_loan\_int\_rate = \sum_{i=1}^N \mathcal{N}(mu\_loan\_int\_rate,\,std\_loan\_int\_rate) & \text{otherwise}.
\end{cases}

$\text{loan_status} = \prod_{i=1}^{N} \mathcal B \left[ {\sigma(logits_i)} \right]$

In [ ]:
def model(data: pd.DataFrame, y: pd.Series | None = None):
    b_person_age = pyro.sample("b_person_age", dist.Normal(0, 1))
    b_person_income = pyro.sample("b_person_income", dist.Normal(0, 1))
    b_loan_amnt = pyro.sample("b_loan_amnt", dist.Normal(0, 1))
    b_loan_int_rate = pyro.sample("b_loan_int_rate", dist.Normal(0, 1))

    logits = b_person_age * torch.tensor(data["person_age"].values)
    logits += b_person_income * torch.tensor(data["person_income"].values)
    logits += b_loan_amnt * torch.tensor(data["loan_amnt"].values)

    mu_loan_int_rate = pyro.sample("mu_loan_int_rate", dist.Normal(0, 1))
    std_loan_int_rate = pyro.sample("std_loan_int_rate", dist.HalfNormal(1))

    impute_loan_int_rate = pyro.sample(
        "impute_loan_int_rate", dist.Normal(
            mu_loan_int_rate,
            std_loan_int_rate
        ).mask(False)
    )

    loan_int_rate = torch.tensor(data["loan_int_rate"].values)
    loan_int_rate = torch.where(torch.isnan(loan_int_rate),
                                impute_loan_int_rate,
                                loan_int_rate)


    logits += b_loan_int_rate * loan_int_rate

    b_person_home_ownership = pyro.sample("b_person_home_ownership",
                                          dist.Normal(0, 1).expand([4]))
    b_loan_intent = pyro.sample("b_loan_intent", dist.Normal(0, 1).expand([6]))
    b_cb_person_default_on_file = pyro.sample("b_cb_person_default_on_file",
                                              dist.Normal(0, 1).expand([2]))

    logits += b_person_home_ownership[data["person_home_ownership"].to_numpy()]
    logits += b_loan_intent[data["loan_intent"].to_numpy()]
    logits += b_cb_person_default_on_file[
        data["cb_person_default_on_file"].to_numpy()]

    obs = None if y is None else torch.tensor(y.values, dtype=torch.float64)

    for i in pyro.plate("N", len(data.index)):
        pyro.sample("loan_int_rate", dist.Normal(mu_loan_int_rate,
                                                 std_loan_int_rate
                                                ), obs=loan_int_rate)
        return pyro.sample("loan_status",
                           dist.Bernoulli(logits=logits),
                           obs=obs)

Визуализируем графическую вероятностную модель, соответствующую совместному распределению данных и весов.

In [ ]:
pyro.render_model(model,
                  model_args=(df_train, df_train["loan_status"]),
                  render_distributions=True,
                  render_params=True)

Выполним **инференс модели** с использованием марковских последовательностей Монте-Карло (MCMC).

Данная процедура выполняет нахождение апостериорного распределения весов модели, многократно сэмплируя из совместного распределения и приводя распределение сэмплов к апостериорному распределению весов.

Процедура сэмплирования может быть __достаточно длительной__.

In [ ]:
num_of_samples = 200

mcmc = MCMC(NUTS(model, jit_compile=True), num_samples=num_of_samples)
mcmc.run(df_train, df_train["loan_status"])
mcmc.summary()

После выполнения процедуры сэмплирования проверьте значения "r_hat" в сводке параметров. Они должны быть близкими к единице, что указывает на сходимость метода.

Получить сэмплы из апостериорного распределения весов можно следующим образом:

In [ ]:
posterior_samples = mcmc.get_samples()
posterior_samples

После выполнения инференса займёмся процедурой __предсказания__ с использованием обученной модели.

Загрузим датасет с тестовыми данными и посмотрим на него.

In [ ]:
df_predict_name = "/content/credit_risk_dataset_predict.csv"
df_predict = pd.read_csv(df_predict_name)
df_predict

Выполним над тестовым датасетом те же преобразования признаков, что и над тренировочным:

In [ ]:
df_predict = column_transformer.transform(df_predict)
df_predict

Получим распределение наблюдаемых переменных для каждого объекта из тестовой выборки с использованием апостериорных распределений весов (полученных после обучения модели).

In [ ]:
posterior_predictive = Predictive(model, posterior_samples)(df_predict)
posterior_predictive

Получим распределение наблюдаемых переменных для каждого объекта из тестовой выборки с использованием априорных распределений весов (полученных до обучения модели).

In [ ]:
prior = Predictive(model, num_samples=num_of_samples)(df_predict)
prior

Сформируем данные для визуализации с помощью библиотеки [ArViZ](https://python.arviz.org/en/stable/).

In [ ]:
pyro_data = az.from_pyro(
    mcmc, prior=prior, posterior_predictive=posterior_predictive
)
pyro_data

Визуализируем апостериорные распределения всех латентных переменных (весов модели).

In [ ]:
az.plot_posterior(pyro_data)

Визуализируем семейство распределений латентных переменных, полученных моделью, и сравним их с эмпирическим распределением, сформированным из тестового датасета.

In [ ]:
az.plot_ppc(pyro_data)

Посмотрим на апостериорные распределения латентных переменных (весов модели) всесте с "трассой" -- набором сэмплов, полученных в процессе выполнения инференса методом MCMC.

In [ ]:
az.plot_trace(pyro_data)

Визуализируем распределение переменной статуса кредита (целевая бинарная переменная, показывающая, возвращен ли клиентом кредит) для одного случайного клиента. Красной пунктирной линией показано истинное значение целевой переменной у данного объекта.

In [ ]:
df_predict_one = df_predict.sample(n=1)

posterior_predictive_one = Predictive(model, posterior_samples)(df_predict_one)
ser = pd.Series(
    posterior_predictive_one["loan_status"].detach().numpy().squeeze()
)
print("Ground True value: ", df_predict_one["loan_status"].values.squeeze())
print("Probability of label to be one: ", ser.mean())
print("Distribution:")
result = plt.hist(ser, color='c', edgecolor='k', alpha=0.65)
plt.axvline(ser.mean(), color='r', linestyle='dashed', linewidth=1)

### Индивидуальное задание

Опираясь на представленное исследование и используя его в качестве образца, **выполните аналогичные действия с выбранным Вами датасетом**.

В процессе работы:

1. Выберите датасет, содержащий табличные данные. Датасеты можно найти, например, на [Kaggle](https://www.kaggle.com/datasets) и [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). __Хотя бы один столбец__ датасета должен содержать пропуски.
2. Выполните __разведочный анализ__ датасета, выполните требуемые __преобразования признаков__.
3. Разделите датасет на __обучающую__ и __тестовую__ части.
4. Поставьте задачу __классификации__ или __регрессии__ для данных из датасета.
5. Составьте __байесовскую модель__ для решения поставленной задачи и определения пропущенных данных. Запишите совместное распределение наблюдаемых и латентных переменных (данных и весов соответственно).
6. __Запрограммируйте__ модель с использованием библиотеки Pyro. Получите изображение __графической вероятностной модели__, сформированной Pyro.
7. __Обучите__ вероятностную модель с использованием MCMC.
8. __Визуализируйте__ апостериорные распределения весов модели и "трассы", полученные в процессе сэмплирования. Критически оцените полученный результат.
9. __Визуализируйте__ эмпирические распределения латентных переменных из тестовой части датасета и сравните их с семейством предиктивных распределений, полученных моделью. Критически оцените полученный результат.
10. Для произвольного объекта из тестовой части датасета получите __предиктивное распределение__ наблюдаемых переменных и соотнесите его с точным значением соответствующих параметров рассматриваемого объекта.
11. Сделайте __выводы__ по работе.

### Список вопросов для подготовки к отчету

1. Основные понятия теории вероятностей: плотность распределения, правило нормировки, совместная плотность вероятности, математическое ожидание, дисперсия, условная плотность вероятности. Условная независимость.
2. Правило произведения для плотности совместного распределения.
3. Правило суммирования. Маргинализация случайных величин.
4. Обращение условного распределения. Теорема Байеса.
5. Сущность байесовского распределения. Примеры.
6. Понятие графических вероятностных моделей. Представление вероятностных отношений в виде графа. Байесовская сеть.
7. Наблюдаемые и латентные величины, их обозначения в графических моделях.
8. Понятие d-разделимости. Последовательная, расходящаяся и сходящаяся связи. Explaining away.
9. Нотация "планок" в графических моделях. Примеры.
10. Основные понятия классического машинного обучения: объекты, признаки, метки, целевая переменная, модель, функции ошибки, эмпирический риск. Принцип минимума эмпирического риска.
11. Классический и байесовский подходы к машинному обучению, их сравнительный анализ. Пример линейной регрессии с точки зрения обоих подходов.
12. Минимизация эмпирического риска как максимизация правдоподобия. Принципы максимальных правдоподобия и апостериорной вероятности.
13. Частотный и байесовский подходы к описанию событий в ML.
14. Постановка задачи и основная проблема байесовского инференса, способы её решения. Сопряженные распределения.
15. Методы Монте-Карло, их эффективность при решении интегралов. Сравнение с аналитическим и численным решениями. Базовый эстиматор Монте-Карло.
16. Понятие семплирования случайной величины. Способы семплирования. Теорема об обратной функции распределения и её применение для семплирования.
17. Семплирование с отклонением (Rejection sampling): алгоритм, его корректность и применимость. Свойства семплирования с отклонением.
18. Семплирование по важности (Importance sampling): алгоритм, его корректность и применимость. Forward sampling.
19. Фреймворк для вероятностного программирования Pyro. Краткая характеристика, построение вероятностных моделей с его помощью.
20. Сущность и реализация сторонних эффектов функций в Pyro. Понятие "трасс". Библиотека Poutine. Обработчики сторонных эффектов (handlers). Примеры.

**Данный список вопросов не является окончательным, он будет дополняться вопросами по мере прохождения соответствующего материала!**





### Список литературы для подготовки к отчету

1. Дауни, А. Б. Байесовские модели / А. Б. Дауни ; перевод с английского В. А. Яроцкого. — Москва : ДМК Пресс, 2018. — 182 с. URL: https://e.lanbook.com/book/131695 (дата обращения: 13.02.2024). — Режим доступа: для авториз. пользователей.

2. Barber D. Bayesian Reasoning and Machine Learning. — Cambridge University Press, 2012. URL: [http://www0.cs.ucl.ac.uk/staff/d.barber/brml/](http://www0.cs.ucl.ac.uk/staff/d.barber/brml/) (дата обращения: 13.02.2024).

3. Bishop M.C. Pattern Recognition and Machine Learning. — Springer, 2006. — 738 с. URL: [https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf) (дата обращения: 13.02.2024).

4. Pyro: официальная документация. 2024. URL: [https://pyro4ci.readthedocs.io/en/latest/](https://pyro4ci.readthedocs.io/en/latest/) (дата обращения: 13.02.2024).